In [1]:
#The spatial gene expression dataset available on 10x genomics wesbite provides following files: 
#(i) probe file with the description of probes and their target genes, 
#(ii) barcoded BAM file with the information of reads alignment,
#(iii) feature-barcode matrices used for secondary analysis,
#(iv) spatial imaging data that describes spot barcode locations in the tissue section images

## Process the SAM file

In [ ]:
#Convert the BAM to SAM file
#From the SAM file we need to keep only the reads belonging to in-tissue spots

In [ ]:
#To find out the list of spot barcodes that contain tissue:

# Read the list of barcodes from "barcodes.tsv"
barcodes_set = set()
with open("/work/FAC/FBM/DBC/cdessim2/default/amaurya/CytAssist_FF_HBC/filtered_feature_bc_matrix/barcodes.tsv", "r") as barcode_file:
    for line in barcode_file:
        barcode = line.strip()
        barcodes_set.add(barcode)

# Filter and write the matching lines to a new SAM file
with open("/work/FAC/FBM/DBC/cdessim2/default/amaurya/CytAssist_FF_HBC/CytAssist_Fresh_all.sam", "r") as sam_file, open("/work/FAC/FBM/DBC/cdessim2/default/amaurya/CytAssist_FF_HBC/CytAssist_Fresh_in_tissue.sam", "w") as output_file:
    for line in sam_file:
        if "CB:Z:" in line:
            fields = line.strip().split("\t")
            for field in fields:
                if field.startswith("CB:Z:"):
                    barcode = field[5:]
                    if barcode in barcodes_set:
                        output_file.write(line)
                        break  # No need to check further fields for this line

In [2]:
#To filter out the reads that donot belong to in-tissue spots:

#Read the list of barcodes from "barcodes.tsv"
barcodes_set = set()
with open("/work/FAC/FBM/DBC/cdessim2/default/amaurya/CytAssist_FFPE_melanoma/input_gene_data/barcodes.tsv", "r") as barcode_file:
    for line in barcode_file:
        barcode = line.strip()
        barcodes_set.add(barcode)

# Filter and write the matching lines to a new SAM file
with open("/work/FAC/FBM/DBC/cdessim2/default/amaurya/CytAssist_FFPE_melanoma/CytAssist_FFPE_Human_Skin_Melanoma_possorted_genome_sam.sam", "r") as sam_file, open("/work/FAC/FBM/DBC/cdessim2/default/amaurya/CytAssist_FFPE_melanoma/CytAssist_FFPE_Melanoma_in_tissue.sam", "w") as output_file:
    for line in sam_file:
        if "CB:Z:" in line:
            fields = line.strip().split("\t")
            for field in fields:
                if field.startswith("CB:Z:"):
                    barcode = field[5:]
                    if barcode in barcodes_set:
                        output_file.write(line)
                        break  # No need to check further fields for this line

#reads from all spots = 193483397
#reads from in-tissue spots = 186179197

In [ ]:
#Keep only reads that have MAPQ (mapping quality) 255 and SAM flag 0 or 16. Remove the rest.
#Done on command line:
#awk '$5 == 255 && ($2 == "0" || $2 == "16") {print}' CytAssist_FFPE_Melanoma_in_tissue.sam > CytAssist_FFPE_Melanoma_in_tissue_fr.sam
#fr stands for filtered reads
#After this filtering, 111411063 reads remain. (Initally, in tissue reads were 186179197)

In [3]:
#Then keep only those reads targeted by TRUE, sp+unsp+misid probes (filtered probes)
#The following code chunk is to remove the reads from FALSE, DEPRECATED probes.
#For this we can use the file "Exon/ALL_filt_probes_targeting_exons_col1" (53219 probes)
#this file is made by taking first column of the file "ALL_filt_probes_targeting_exons"
#in this list we don't consider 249 filt true probes because they don't have target exons

## Read probe IDs from the CSV file
csv_probe_ids = set()
with open('/work/FAC/FBM/DBC/cdessim2/default/amaurya/Exons/ALL_filt_probes_targeting_exons_col1', 'r') as csv_file:
    next(csv_file)  # Skip the header row
    for line in csv_file:
        probe_id = line.strip()
        csv_probe_ids.add(probe_id)

# Open the SAM files for reading and writing
with open('/work/FAC/FBM/DBC/cdessim2/default/amaurya/CytAssist_FFPE_melanoma/CytAssist_FFPE_Melanoma_in_tissue_fr.sam', 'r') as input_file, open('/work/FAC/FBM/DBC/cdessim2/default/amaurya/CytAssist_FFPE_melanoma/CytAssist_FFPE_Melanoma_in_tissue_016fr_trueprobe.sam', 'w') as output_file:
    for line in input_file:
        if line.startswith('@'):  # Preserve header lines
            output_file.write(line)
        else:
            fields = line.split('\t')
            for field in fields:
                if field.startswith('pr:Z:'):
                    probe_id = field.split(':')[2]
                    if probe_id in csv_probe_ids:
                        output_file.write(line)
                        break  # Found a matching probe ID, no need to check other fields

# Close the files
input_file.close()
output_file.close()

## Generate the probe count matrix

In [ ]:
#After selecting the desired probes, create the probe count matrix
#the following chunk counts the reads of each probe in each barcode

In [4]:
# Initialize a dictionary to store probe_id counts for each spot_barcode
matrix_data = {}

# Path to SAM file
with open('/work/FAC/FBM/DBC/cdessim2/default/amaurya/CytAssist_FFPE_melanoma/CytAssist_FFPE_Melanoma_in_tissue_016fr_trueprobe.sam', 'r') as sam_file:
    for line in sam_file:
        fields = line.strip().split('\t')
        
        # Extract probe_id and spot_barcode
        probe_id = None
        spot_barcode = None
        for field in fields:
            if field.startswith("pr:Z:"):
                probe_id = field.split(':')[2]
            elif field.startswith("CB:Z:"):
                spot_barcode = field.split(':')[2]
                break
        
        # Update the matrix_data dictionary
        if probe_id and spot_barcode:
            if probe_id not in matrix_data:
                matrix_data[probe_id] = {}
            if spot_barcode not in matrix_data[probe_id]:
                matrix_data[probe_id][spot_barcode] = 0
            matrix_data[probe_id][spot_barcode] += 1

# Extract unique spot_barcodes and probe_ids
unique_spot_barcodes = sorted(set(barcode for data in matrix_data.values() for barcode in data.keys()))
unique_probe_ids = sorted(matrix_data.keys())

# Create the matrix
matrix = []
for probe_id in unique_probe_ids:
    row = [matrix_data[probe_id].get(barcode, 0) for barcode in unique_spot_barcodes]
    matrix.append(row)
    
# Save the matrix to a file
output_file_path = '/work/FAC/FBM/DBC/cdessim2/default/amaurya/CytAssist_FFPE_melanoma/probe_count_matrix'
with open(output_file_path, 'w') as matrix_file:
    # Write the header (spot_barcodes)
    matrix_file.write("\t" + "\t".join(unique_spot_barcodes) + "\n")

    # Write the matrix data
    for probe_id, row in zip(unique_probe_ids, matrix):
        matrix_file.write(probe_id + "\t" + "\t".join(map(str, row)) + "\n")

## Create all chr exon matrix

In [2]:
#Now, replace the probe_id by its matching exon_id (creating a new line for each exon_id)

In [ ]:
#The following chunk replaces the probe_ids by corresponding exon_ids. 
#Two probe_ids may target same exon_ids so this chunk will replace the both probe with same exon_ids
#Therefore, the ouput will contain duplicate lines

In [1]:
import csv

# Step 1: Read the probe-exon mapping file and create a dictionary
probe_to_exon = {}
with open("/work/FAC/FBM/DBC/cdessim2/default/amaurya/Exons/ALL_filt_probes_targeting_exons", "r") as mapping_file:
    for line in mapping_file:
        parts = line.strip().split("\t")
        probe_id = parts[0].strip().lower()  # Convert to lowercase and strip white spaces #this is imp
        exon_ids = parts[1].split(";")
        probe_to_exon[probe_id] = exon_ids

# Step 2: Process the count matrix line by line
with open("/work/FAC/FBM/DBC/cdessim2/default/amaurya/CytAssist_FFPE_melanoma/probe_count_matrix", "r") as count_matrix_file, open("/work/FAC/FBM/DBC/cdessim2/default/amaurya/CytAssist_FFPE_melanoma/exon_count_matrix_with_dup", "w", newline="") as new_matrix_file:
    count_matrix_reader = csv.reader(count_matrix_file, delimiter="\t")
    new_matrix_writer = csv.writer(new_matrix_file, delimiter="\t")

    for row in count_matrix_reader:
        probe_id = row[0].strip().lower()  # Convert to lowercase and strip white spaces
        counts = row[1:]
        
        if probe_id in probe_to_exon:
            exon_ids = probe_to_exon[probe_id]
            for exon_id in exon_ids:
                new_matrix_writer.writerow([exon_id] + counts)
        else:
            new_matrix_writer.writerow([probe_id] + counts)

In [ ]:
#the file "exon_count_matrix_with_dup" can be deleted later
#sum up the counts of duplicate exon_ids

In [2]:
import csv

# Step 1: Read the modified count matrix and create a dictionary to store counts for each unique exon ID
exon_counts = {}

with open("/work/FAC/FBM/DBC/cdessim2/default/amaurya/CytAssist_FFPE_melanoma/exon_count_matrix_with_dup", "r") as modified_matrix_file:
    modified_matrix_reader = csv.reader(modified_matrix_file, delimiter="\t")
    barcode_line = next(modified_matrix_reader)  # Store the barcode line

    for row in modified_matrix_reader:
        exon_id = row[0].strip()          #strip white spaces #IMPORTANT
        counts = list(map(int, row[1:]))  #Convert counts to integers
        if exon_id in exon_counts:
            # If the exon ID is already in the dictionary, add the counts
            exon_counts[exon_id] = [x + y for x, y in zip(exon_counts[exon_id], counts)]
        else:
            # If the exon ID is not in the dictionary, initialize it with the counts
            exon_counts[exon_id] = counts

# Step 2: Write the merged count matrix with summed counts, including the barcode line
with open("/work/FAC/FBM/DBC/cdessim2/default/amaurya/CytAssist_FFPE_melanoma/input_exon_data/exon_count_matrix", "w", newline="") as merged_matrix_file:
    merged_matrix_writer = csv.writer(merged_matrix_file, delimiter="\t")
    merged_matrix_writer.writerow(barcode_line)  # Write the barcode line

    for exon_id, counts in exon_counts.items():
        merged_matrix_writer.writerow([exon_id] + list(map(str, counts)))

print("Merged count matrix with summed counts has been saved to exon_count_matrix.txt")

Merged count matrix with summed counts has been saved to exon_count_matrix.txt


## Prepare STARCH input files

In [ ]:
# Preparing features.tsv for exons: (we need exons present in count matrix)
# Extract the features (exon_ids) i.e the first field of all lines
# cut -f 1 exon_count_matrix > f1.tsv
# there might be an empty first line, check and delete if needed: sed -i 1d
# Create a duplicate of this column →  awk -F'\t' '{print $1 "\t" $1}' f1.tsv > f2.tsv
# Add a third field “Exon Expression” to each row → awk 'BEGIN {OFS="\t"} {$3="Exon Expression"; print}' f2.tsv > features.tsv


In [ ]:
# For preparing hgtable for exon matrix:
# First print an empty line: echo “” > file.txt
# Then add numbers starting from zero: for i in {0..261184}; do echo $i >> file.txt; done
# (261184 is one less than total)
# In exon id file add a header row: echo -e "chrom\tcdsStart\tcdsEnd\tname2" > output.txt
# cat unique_exon_ids_coordinates.txt >> output.txt
# Merge the two files: paste file.txt output.txt > merged_output.txt


## Convert to matrix.mtx

In [4]:
import numpy as np
import scipy.io as sio

In [ ]:
#Make a features.tsv file (a numbered list of exons in count matrix)
#cut -f 1 exon_count_matrix > features1.tsv (use sed -i 1d if blank line at top)
#awk 'BEGIN{OFS="\t"} {print NR, $0}' features1.tsv > features_with_serial.tsv

In [5]:
exon_ids = []  # List to store exon IDs
barcode_ids = []  # List to store barcode IDs

# Load exon IDs from the exon_id.tsv file
with open('/work/FAC/FBM/DBC/cdessim2/default/amaurya/CytAssist_FFPE_melanoma/input_exon_data/features_with_serial.tsv', 'r') as f:
    for line in f:
        exon_ids.append(line.strip())

# Load barcode IDs from the barcode_id.tsv file
with open('/work/FAC/FBM/DBC/cdessim2/default/amaurya/CytAssist_FFPE_melanoma/input_exon_data/barcodes_with_serial.tsv', 'r') as f:
    for line in f:
        barcode_ids.append(line.strip())

In [ ]:
#The input matrix below should not have row and column headers. Just the counts.
#Remove the first row: awk 'NR > 1' exon_count_matrix > new_matrix
#Remove the first col: cut -f 2- < new_matrix > output_matrix

In [6]:
count_matrix = np.genfromtxt('/work/FAC/FBM/DBC/cdessim2/default/amaurya/CytAssist_FFPE_melanoma/input_exon_data/output_matrix', delimiter='\t', dtype=int)

In [7]:
transposed_matrix = np.transpose(count_matrix)

In [8]:
from scipy.sparse import coo_matrix

# Create a COO (Coordinate) matrix
sparse_matrix = coo_matrix(transposed_matrix)

In [9]:
# scipy's mmwrite function to save the sparse matrix in Matrix Market format
sio.mmwrite('/work/FAC/FBM/DBC/cdessim2/default/amaurya/CytAssist_FFPE_melanoma/input_exon_data/matrix_v1.mtx', sparse_matrix)

In [ ]:
#The matrix_v1.mtx generated has the fields in incorrect order. So need to interchange the first and second field.
# awk '{print $2, $1, $3}' matrix_v1.mtx > matrix.mtx

In [ ]:
#However, the two headers of matrix.mtx generated here do not match the 10x output matrix.
#So, need to modify the matrix.mtx headers
# First, delete first header: sed -i '1d' matrix.mtx
#Then add desired first header: sed -i '1s/^/%%MatrixMarket matrix coordinate integer general\n/' matrix.mtx
#Next, delete second header: sed -i '2d' matrix.mtx
#Then add desired second header: sed -i '2s/^/%metadata_json: {"software_version": "spaceranger-2.0.1", "format_version": 2}\n/' matrix.mtx